In [144]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, Lasso, LinearRegression
from sklearn.metrics import accuracy_score, f1_score

import warnings
warnings.simplefilter('ignore', category=FutureWarning)

In [145]:
train = pd.read_csv('data/train.tsv', names=[f'x{i}' for i in range(100)]+['y'], sep='\t')
test = pd.read_csv('data/test.tsv', names=[f'x{i}' for i in range(100)], sep='\t')

In [146]:
X = train.drop('y', axis=1)
y = train['y']

In [149]:
train_X, val_X, train_y, val_y = train_test_split(X, y, test_size=0.2, random_state=0, stratify=y)

In [127]:
# from itertools import combinations
# features = train_X.columns
# pairs = combinations(features, 20)

In [132]:
from catboost import CatBoostClassifier, Pool

In [154]:
cb = CatBoostClassifier(verbose=50, iterations=200)
cb.fit(train_X, train_y, eval_set=(val_X, val_y))

Learning rate set to 0.193301
0:	learn: 0.6907501	test: 0.6941119	best: 0.6941119 (0)	total: 26.5ms	remaining: 5.28s
50:	learn: 0.0136847	test: 0.0155424	best: 0.0155154 (47)	total: 1.35s	remaining: 3.96s
100:	learn: 0.0034603	test: 0.0043414	best: 0.0043406 (91)	total: 2.66s	remaining: 2.6s
150:	learn: 0.0020881	test: 0.0028061	best: 0.0028016 (128)	total: 3.93s	remaining: 1.27s
199:	learn: 0.0015929	test: 0.0022611	best: 0.0022582 (189)	total: 5.15s	remaining: 0us

bestTest = 0.002258242774
bestIteration = 189

Shrink model to first 190 iterations.


In [156]:
(cb.predict(train_X) == train_y).mean()

1.0

In [157]:
(cb.predict(val_X) == val_y).mean()

1.0

In [159]:
pd.Series(cb.predict(test).astype(int)).to_csv('predictions.csv', index=False, header=False)

In [161]:
imp = cb.get_feature_importance(Pool(train_X, label=train_y))

In [163]:
np.argsort(imp)[::-1][:10]

array([95,  5, 65, 61, 87, 72, 85, 98, 41, 97], dtype=int64)

In [166]:
np.sort(imp)[::-1][:10]

array([4.87742828e+01, 4.87159252e+01, 5.48633495e-02, 4.85416188e-02,
       4.67495636e-02, 4.27249082e-02, 4.23191535e-02, 4.20191937e-02,
       4.19529881e-02, 4.07480253e-02])